In [1]:
import paramiko
import pandas as pd
import os
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

ModuleNotFoundError: No module named 'pandas'

In [ ]:
load_dotenv(find_dotenv(), override=True)

In [30]:
connection = "mysql+pymysql://" + os.getenv("DB_USER_NAME") + ":" + os.getenv("DB_PASSWORD") + "@127.0.0.1/DublinBus"

In [33]:
engine = create_engine(connection)

In [43]:
def get_dataframe(route):
    """
    Input: route number
    Output: Dataframe for that route number
    """
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.connect(os.getenv("SSH_ADDRESS"), 22, "student", os.getenv("SSH_PASSWORD"))
    sftp_client = client.open_sftp()
    file_name = "/home/student/data/data/routes/" + str(route) + "r.csv"
    remote_file = sftp_client.open(file_name)
    return pd.read_csv(remote_file, parse_dates=['DAYOFSERVICE'])

In [44]:
def get_distance_txt():
    """
    Input: route number
    Output: Dataframe for that route number
    """
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.connect(os.getenv("SSH_ADDRESS"), 22, "student", os.getenv("SSH_PASSWORD"))
    sftp_client = client.open_sftp()
    file_name = "/home/student/data/data/routes/distance.txt"
    remote_file = sftp_client.open(file_name)
    return  pd.read_csv(remote_file,delimiter=' ', header=None, names=['LineID', 'PLANNEDTIME_DEP', 'STOPPOINTID', 'PROGRNUMBER', 'DISTANCE_FROM_START'])

In [45]:
df = get_dataframe("41D")

In [16]:
distance = get_distance_txt()

/home/jmcl/anaconda3/envs/LinearRegression/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [21]:
for dist in distance:
    print(dist)

LineID
PLANNEDTIME_DEP
STOPPOINTID
PROGRNUMBER
DISTANCE_FROM_START


In [25]:
df.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LineID,RouteID,Direction
0,2018-12-11,8173538,13,3699,34080,34080,33421,33421,41D,41D_27,2
1,2018-12-11,8173538,12,3698,34020,34020,33407,33407,41D,41D_27,2
2,2018-12-11,8173538,11,3697,33960,33960,33369,33369,41D,41D_27,2
3,2018-12-11,8173538,10,3696,33924,33924,33346,33346,41D,41D_27,2
4,2018-12-11,8173538,9,3695,33870,33870,33301,33301,41D,41D_27,2


In [46]:
route="41D"

#day, weekday, month
df['day']=df['DAYOFSERVICE'].dt.dayofweek
df['weekday']=1
df.loc[df['day'].isin([5,6]), 'weekday']=0
df['month']=df['DAYOFSERVICE'].dt.month #school term

#CALCULATE PERCENTAGE OF JOURNEY COMPLETED
trips=df.groupby(['DAYOFSERVICE','TRIPID'])['PROGRNUMBER'].unique().reset_index(level=0)
trips=trips.reset_index(level=0)
trips['PROGRNUMBER']=trips['PROGRNUMBER'].apply(sorted)
trips['TOTALSTOPS']=trips.PROGRNUMBER.apply(lambda x: max(x))
trips=trips.drop('PROGRNUMBER', 1)
df=pd.merge(df, trips, on=['DAYOFSERVICE','TRIPID'] )
df['PERCENTSTOPSCOMPLETED']=(df['PROGRNUMBER']/df['TOTALSTOPS'])*100
df=df.drop('TOTALSTOPS',1)

#Merge with historical weather
df['TIMESTAMP']=pd.to_datetime(df.DAYOFSERVICE)+pd.to_timedelta(df.ACTUALTIME_ARR, unit='second')
#merge weather
min_date=df.DAYOFSERVICE.min()
max_date=df.DAYOFSERVICE.max()
sql='Select * from HistoricalWeather where Timestamp >= "'+str(min_date)+'" and Timestamp <= "'+str(max_date)+'";'
df1=pd.read_sql_query(sql, con=engine, parse_dates={'DayOfService':'%Y-%m-%d %H:%M:%S'})
df=df.sort_values('TIMESTAMP')
df=pd.merge_asof(df, df1, left_on='TIMESTAMP', right_on='Timestamp', direction='backward')

#Add Peak/Off Peak
df['PEAK']=0
df.loc[(df.weekday==1)&(((df.TIMESTAMP.dt.hour>=7) & (df.TIMESTAMP.dt.hour<=9)) | ((df.TIMESTAMP.dt.hour>=16) & (df.TIMESTAMP.dt.hour<=19))),'PEAK']=1
df=df.drop('TIMESTAMP',1)

#Distance from depo
df.PLANNEDTIME_DEP=pd.to_timedelta(df.PLANNEDTIME_DEP, unit='seconds')
df['DISTANCE_FROM_START']=None
distance.loc[(distance.LineID==route) | (distance.LineID==route.lower())]
distance.drop('LineID',1)
distance.PLANNEDTIME_DEP=pd.to_timedelta(distance.PLANNEDTIME_DEP, unit='seconds')
df=pd.merge(df, distance, on=['PLANNEDTIME_DEP', 'STOPPOINTID','PROGRNUMBER'], how='left')
df['DISTANCE_FROM_START'] = df['DISTANCE_FROM_START_y'].fillna(df['DISTANCE_FROM_START_x'])
df=df.drop(['DISTANCE_FROM_START_x', 'DISTANCE_FROM_START_y'], axis=1)
#drop any null values are some stops are no longer being served by DB
df=df.drop(df.loc[df.DISTANCE_FROM_START.isnull()].index, 0)

#Add in national hols
nathols=['2018-01-01', '2018-03-17',  '2018-03-30',  '2018-04-01',  '2018-04-02',  '2018-05-07',  '2018-06-04',
 '2018-08-06',  '2018-10-29',  '2018-12-21',  '2018-12-24', '2018-12-25', '2018-12-26', '2018-12-31']
df['OBSERVENCE/NATIONALHOL']=0
df.loc[df.DAYOFSERVICE.isin(nathols), 'OBSERVENCE/NATIONALHOL']=1

#Target Feature
df['DIFFERENCE_PLANNED_ACTUAL_ARR']=df['ACTUALTIME_ARR']-df['PLANNEDTIME_ARR']

#drop all unneccessary features
df=df.drop(['Timestamp', 'TRIPID', 'LineID_y'], 1)

In [48]:
df.head()

,DAYOFSERVICE,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LineID_x,RouteID,Direction,...,temperature,windSpeed,sunshineAmount,cloudAmount,currentWeatherDesc,pastHourWeatherDesc,PEAK,DISTANCE_FROM_START,OBSERVENCE/NATIONALHOL,DIFFERENCE_PLANNED_ACTUAL_ARR
0,2018-12-11,1,288,27000,07:30:00,27063,27063,41D,41D_26,1,...,6.8,4,0.0,7,10,22,1,0,0,63
1,2018-12-11,1,288,27000,07:30:00,27063,27063,41D,41D_26,1,...,6.8,4,0.0,7,10,22,1,0,0,63
2,2018-12-11,1,288,27000,07:30:00,27063,27063,41D,41D_26,1,...,6.8,4,0.0,7,10,22,1,0,0,63
3,2018-12-11,2,1171,27042,07:30:42,27127,27147,41D,41D_26,1,...,6.8,4,0.0,7,10,22,1,189.195,0,85
4,2018-12-11,2,1171,27042,07:30:42,27127,27147,41D,41D_26,1,...,6.8,4,0.0,7,10,22,1,189.195,0,85
